In [1]:
import pandas as pd
import numpy as np
import warnings
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 100)
pd.options.display.max_colwidth = 1000
warnings.filterwarnings("ignore")

In [44]:
train = pd.DataFrame([['Sayan',float('Nan'),float('Nan'),190, 90,1],
                        ['Batima','F','Kostanay',162,54,0],
                        ['Nadira','F','Astana',165,50,0],
                        ['Zhanbolat','M','Aqtobe',179,70,1],
                        ['Azamat','M','Aqtobe',179,75,1]
                       ],
                       columns = ['NAME', 'SEX', 'CITY','HEIGHT', 'WEIGHT', 'TARGET'])
test = pd.DataFrame([['Madiyar','M','Kostanay',185, 72,''],
                        ['Marat','M','Pavlodar',180,float('Nan'),''],
                        ['Dair','M','Karaganda',184,80,''],
                        ['Madina','F',float('Nan'),
                         float('Nan')
                         ,52,''],
                        ['Perizat','F','Pavlodar',165,50,'']
                       ],
                       columns = ['NAME', 'SEX', 'CITY','HEIGHT', 'WEIGHT', 'TARGET'])

### Dummy - кодирование

In [112]:
class GetDummies():
    def __init__(self,  part=0.01):
        self.part = part
        self.unique_values = {}
        self.cat_columns = []
        self.num_columns = []

    def fit(self, df):
        self.cat_columns = list(df.columns[df.dtypes == 'object'])
        self.num_columns = list(df.columns[df.dtypes == 'int']) + list(df.columns[df.dtypes == 'float'])
        for i in self.cat_columns:
            tmp = df[i].value_counts(normalize=True).reset_index()
            tmp = tmp[tmp[i] >= self.part]
            self.unique_values[i] = list(tmp['index'].unique())

    def transform(self, df, inplace=False):
        if inplace:
            columns = []
            for i in self.cat_columns:
                for j in range(len(self.unique_values[i])):
                    columns.append(str(i + '=' + str(self.unique_values[i][j])))
                    if i in df.columns:
                        df[i + '=' + str(self.unique_values[i][j])] = \
                        df[i].apply(lambda x: 1 if x == str(self.unique_values[i][j]) else 0)
            return df
        else:
            df_ = df.copy()
            columns = []
            for i in self.cat_columns:
                for j in range(len(self.unique_values[i])):
                    columns.append(str(i + '=' + str(self.unique_values[i][j])))
                    if i in df_.columns:
                        df_[i + '=' + str(self.unique_values[i][j])] = \
                        df_[i].apply(lambda x: 1 if x == str(self.unique_values[i][j]) else 0)
            return df_

    доля по умолчанию

In [113]:
a = GetDummies()
a.fit(train)
a.transform(train)

,NAME,SEX,CITY,HEIGHT,WEIGHT,TARGET,NAME=Batima,NAME=Azamat,NAME=Nadira,NAME=Zhanbolat,NAME=Sayan,SEX=F,SEX=M,CITY=Aqtobe,CITY=Kostanay,CITY=Astana
0,Sayan,NaN,NaN,190,90,1,0,0,0,0,1,0,0,0,0,0
1,Batima,F,Kostanay,162,54,0,1,0,0,0,0,1,0,0,1,0
2,Nadira,F,Astana,165,50,0,0,0,1,0,0,1,0,0,0,1
3,Zhanbolat,M,Aqtobe,179,70,1,0,0,0,1,0,0,1,1,0,0
4,Azamat,M,Aqtobe,179,75,1,0,1,0,0,0,0,1,1,0,0


In [114]:
a.transform(test)

,NAME,SEX,CITY,HEIGHT,WEIGHT,TARGET,NAME=Batima,NAME=Azamat,NAME=Nadira,NAME=Zhanbolat,NAME=Sayan,SEX=F,SEX=M,CITY=Aqtobe,CITY=Kostanay,CITY=Astana
0,Madiyar,M,Kostanay,185.0,72.0,,0,0,0,0,0,0,1,0,1,0
1,Marat,M,Pavlodar,180.0,NaN,,0,0,0,0,0,0,1,0,0,0
2,Dair,M,Karaganda,184.0,80.0,,0,0,0,0,0,0,1,0,0,0
3,Madina,F,NaN,NaN,52.0,,0,0,0,0,0,1,0,0,0,0
4,Perizat,F,Pavlodar,165.0,50.0,,0,0,0,0,0,1,0,0,0,0


    доля более 0.33

In [115]:
a = GetDummies(part=0.333)
a.fit(train)
a.transform(train)

,NAME,SEX,CITY,HEIGHT,WEIGHT,TARGET,SEX=F,SEX=M,CITY=Aqtobe
0,Sayan,NaN,NaN,190,90,1,0,0,0
1,Batima,F,Kostanay,162,54,0,1,0,0
2,Nadira,F,Astana,165,50,0,1,0,0
3,Zhanbolat,M,Aqtobe,179,70,1,0,1,1
4,Azamat,M,Aqtobe,179,75,1,0,1,1


In [116]:
a.transform(test)

,NAME,SEX,CITY,HEIGHT,WEIGHT,TARGET,SEX=F,SEX=M,CITY=Aqtobe
0,Madiyar,M,Kostanay,185.0,72.0,,0,1,0
1,Marat,M,Pavlodar,180.0,NaN,,0,1,0
2,Dair,M,Karaganda,184.0,80.0,,0,1,0
3,Madina,F,NaN,NaN,52.0,,1,0,0
4,Perizat,F,Pavlodar,165.0,50.0,,1,0,0


## Таргет кодирование

    доля по умолчанию

In [117]:
class TargetPercentageEncoding():
    def __init__(self, target_column, part=0.05, filltype='mean'):
        self.part = part
        self.target_column = target_column
        self.fit_dict = {}
        self.cat_columns = []
        self.filltype=filltype
    def fit(self, df):
        self.cat_columns = list(df.columns[df.dtypes == 'object'])
        self.cat_columns_fill = {}
        mean_v = df[self.target_column].mean()
        for i in self.cat_columns:
            tmp = df[[i,self.target_column]].groupby(i)[self.target_column].mean().reset_index()
            tmp_ = df.groupby(i).size().reset_index().rename(columns={i:i, 0 : 'CNT'})
            tmp_['CNT'] = tmp_['CNT'] / tmp_['CNT'].sum()
            tmp = tmp.merge(tmp_)
            tmp = tmp[tmp['CNT'] >= self.part]
            if self.filltype=='mean':
                self.cat_columns_fill[i] = mean_v
            else:
                self.cat_columns_fill[i] = 0
            self.fit_dict[i] = pd.Series(tmp[self.target_column].values,index = tmp[i]).to_dict()
    
    def transform(self, df, inplace = False):
        if inplace:
            for i in self.cat_columns:
                val = self.cat_columns_fill[i]
                df[i+'_TARGET_PERCENTAGE'] = df[i].map(self.fit_dict[i]).fillna(val)
            return df
        else:
            df_ = df.copy()
            for i in self.cat_columns:
                val = self.cat_columns_fill[i]
                df_[i+'_TARGET_PERCENTAGE'] = df_[i].map(self.fit_dict[i]).fillna(val)
            return df_

In [118]:
a = TargetPercentageEncoding(target_column='TARGET', part=0.01)
a.fit(train)
a.transform(train)

,NAME,SEX,CITY,HEIGHT,WEIGHT,TARGET,NAME_TARGET_PERCENTAGE,SEX_TARGET_PERCENTAGE,CITY_TARGET_PERCENTAGE
0,Sayan,NaN,NaN,190,90,1,1,0.6,0.6
1,Batima,F,Kostanay,162,54,0,0,0.0,0.0
2,Nadira,F,Astana,165,50,0,0,0.0,0.0
3,Zhanbolat,M,Aqtobe,179,70,1,1,1.0,1.0
4,Azamat,M,Aqtobe,179,75,1,1,1.0,1.0


In [119]:
a.transform(test)

,NAME,SEX,CITY,HEIGHT,WEIGHT,TARGET,NAME_TARGET_PERCENTAGE,SEX_TARGET_PERCENTAGE,CITY_TARGET_PERCENTAGE
0,Madiyar,M,Kostanay,185.0,72.0,,0.6,1,0.0
1,Marat,M,Pavlodar,180.0,NaN,,0.6,1,0.6
2,Dair,M,Karaganda,184.0,80.0,,0.6,1,0.6
3,Madina,F,NaN,NaN,52.0,,0.6,0,0.6
4,Perizat,F,Pavlodar,165.0,50.0,,0.6,0,0.6


     более 0.33

In [120]:
a = TargetPercentageEncoding(target_column='TARGET', part=0.333)
a.fit(train)
a.transform(train)

,NAME,SEX,CITY,HEIGHT,WEIGHT,TARGET,NAME_TARGET_PERCENTAGE,SEX_TARGET_PERCENTAGE,CITY_TARGET_PERCENTAGE
0,Sayan,NaN,NaN,190,90,1,0.6,0.6,0.6
1,Batima,F,Kostanay,162,54,0,0.6,0.0,0.6
2,Nadira,F,Astana,165,50,0,0.6,0.0,0.6
3,Zhanbolat,M,Aqtobe,179,70,1,0.6,1.0,1.0
4,Azamat,M,Aqtobe,179,75,1,0.6,1.0,1.0


In [121]:
a.transform(test)

,NAME,SEX,CITY,HEIGHT,WEIGHT,TARGET,NAME_TARGET_PERCENTAGE,SEX_TARGET_PERCENTAGE,CITY_TARGET_PERCENTAGE
0,Madiyar,M,Kostanay,185.0,72.0,,0.6,1,0.6
1,Marat,M,Pavlodar,180.0,NaN,,0.6,1,0.6
2,Dair,M,Karaganda,184.0,80.0,,0.6,1,0.6
3,Madina,F,NaN,NaN,52.0,,0.6,0,0.6
4,Perizat,F,Pavlodar,165.0,50.0,,0.6,0,0.6


# ValueCountEncoding

In [122]:
class ValueCountEncoding():
    def __init__(self):
        self.fit_dict = {}
        self.cat_columns = []
        
    def fit(self, df):
        self.cat_columns = list(df.columns[df.dtypes == 'object'])
        for i in self.cat_columns:
            tmp = df.groupby(i).size().reset_index().rename(columns={i:i, 0 : 'CNT'})
            tmp['CNT'] = tmp['CNT'] / tmp['CNT'].sum()
            self.fit_dict[i] = pd.Series(tmp['CNT'].values,index = tmp[i]).to_dict()
    
    def transform(self, df, inplace = False):
        if inplace:
            for i in self.cat_columns:
                df[i+'_FREQUENCY'] = df[i].map(self.fit_dict[i]).fillna(0)
            return df
        else:
            df_ = df.copy()
            for i in self.cat_columns:
                df_[i+'_FREQUENCY'] = df_[i].map(self.fit_dict[i]).fillna(0)
            return df_

In [123]:
a = ValueCountEncoding()
a.fit(train)
a.transform(train)

,NAME,SEX,CITY,HEIGHT,WEIGHT,TARGET,NAME_FREQUENCY,SEX_FREQUENCY,CITY_FREQUENCY
0,Sayan,NaN,NaN,190,90,1,0.2,0.0,0.00
1,Batima,F,Kostanay,162,54,0,0.2,0.5,0.25
2,Nadira,F,Astana,165,50,0,0.2,0.5,0.25
3,Zhanbolat,M,Aqtobe,179,70,1,0.2,0.5,0.50
4,Azamat,M,Aqtobe,179,75,1,0.2,0.5,0.50


In [124]:
a.transform(test)

,NAME,SEX,CITY,HEIGHT,WEIGHT,TARGET,NAME_FREQUENCY,SEX_FREQUENCY,CITY_FREQUENCY
0,Madiyar,M,Kostanay,185.0,72.0,,0.0,0.5,0.25
1,Marat,M,Pavlodar,180.0,NaN,,0.0,0.5,0.00
2,Dair,M,Karaganda,184.0,80.0,,0.0,0.5,0.00
3,Madina,F,NaN,NaN,52.0,,0.0,0.5,0.00
4,Perizat,F,Pavlodar,165.0,50.0,,0.0,0.5,0.00


# Fillna

In [190]:
class FillNa():
    def __init__(self, target_column, method='Median'):
        self.fit_dict = {}
        self.cat_columns = []
        self.num_columns = []
        self.target_column = target_column
        self.method = method
        
    def fit(self, df):
        self.cat_columns = list(df.columns[df.dtypes == 'object'])
        self.num_columns = list(set(df.columns) - set(self.cat_columns))
        if set(self.target_column).issubset(self.num_columns):
            self.num_columns = [x for x in self.num_columns if x not in self.target_column]
        else:
            self.cat_columns = [x for x in self.cat_columns if x not in self.target_column]
            
        self.cat_columns_with_na = [x for x in self.cat_columns if df[x].isnull().sum()>0]
        self.num_columns_with_na = [x for x in self.num_columns if df[x].isnull().sum()>0]
        
        for i in self.cat_columns:
            self.fit_dict[i] = df[i].mode()
        if self.method == 'median':
            for i in self.num_columns:
                self.fit_dict[i] = df[i].median()
        elif self.method == 'mean':
            for i in self.num_columns:
                self.fit_dict[i] = df[i].mean()
        elif self.method == 'default':
            for i in self.num_columns:
                self.fit_dict[i] = -99999
            for i in self.cat_columns:
                self.fit_dict[i] = '-99999'
        else:
            return 'Unexpected value of parameter method'
    
    def transform(self, df, inplace=False):
        if inplace:
            for i in self.cat_columns_with_na:
                df[i + '_IS_NAN'] = df[i].isna().astype(int)
                df[i] = df[i].fillna(self.fit_dict[i][0])
            for i in self.num_columns_with_na:
                df[i + '_IS_NAN'] = df[i].isna().astype(int)
                df[i] = df[i].fillna(float(self.fit_dict[i]))
        else:
            df_ = df.copy()
            for i in self.cat_columns:
                if i in self.cat_columns_with_na:
                    df_[i + '_IS_NAN'] = df_[i].isna().astype(int)
                if df[i].isnull().sum()>0:
                    df_[i] = df_[i].fillna(self.fit_dict[i][0])
            for i in self.num_columns:
                if i in self.num_columns_with_na:
                    df_[i + '_IS_NAN'] = df_[i].isna().astype(int)
                if df[i].isnull().sum()>0:
                    df_[i] = df_[i].fillna(float(self.fit_dict[i]))
            return df_

In [186]:
a = FillNa(target_column='TARGET', method='median')
a.fit(train)
a.transform(train)

,NAME,SEX,CITY,HEIGHT,WEIGHT,TARGET,SEX_IS_NAN,CITY_IS_NAN
0,Sayan,F,Aqtobe,190,90,1,1,1
1,Batima,F,Kostanay,162,54,0,0,0
2,Nadira,F,Astana,165,50,0,0,0
3,Zhanbolat,M,Aqtobe,179,70,1,0,0
4,Azamat,M,Aqtobe,179,75,1,0,0


In [187]:
a.transform(test)

,NAME,SEX,CITY,HEIGHT,WEIGHT,TARGET,SEX_IS_NAN,CITY_IS_NAN
0,Madiyar,M,Kostanay,185.0,72.0,,0,0
1,Marat,M,Pavlodar,180.0,70.0,,0,0
2,Dair,M,Karaganda,184.0,80.0,,0,0
3,Madina,F,Aqtobe,179.0,52.0,,0,1
4,Perizat,F,Pavlodar,165.0,50.0,,0,0


In [188]:
a = FillNa(target_column='TARGET', method='default')
a.fit(train)
a.transform(train)

,NAME,SEX,CITY,HEIGHT,WEIGHT,TARGET,SEX_IS_NAN,CITY_IS_NAN
0,Sayan,-,-,190,90,1,1,1
1,Batima,F,Kostanay,162,54,0,0,0
2,Nadira,F,Astana,165,50,0,0,0
3,Zhanbolat,M,Aqtobe,179,70,1,0,0
4,Azamat,M,Aqtobe,179,75,1,0,0


In [189]:
a.transform(test)

,NAME,SEX,CITY,HEIGHT,WEIGHT,TARGET,SEX_IS_NAN,CITY_IS_NAN
0,Madiyar,M,Kostanay,185.0,72.0,,0,0
1,Marat,M,Pavlodar,180.0,-99999.0,,0,0
2,Dair,M,Karaganda,184.0,80.0,,0,0
3,Madina,F,-,-99999.0,52.0,,0,1
4,Perizat,F,Pavlodar,165.0,50.0,,0,0
